In [1]:
"""Example usage:
  python object_detection/dataset_tools/create_oid_tf_record.py \
    --input_annotations_csv=/path/to/input/annotations-human-bbox.csv \
    --input_images_directory=/path/to/input/image_pixels_directory \
    --input_label_map=/path/to/input/labels_bbox_545.labelmap \
    --output_tf_record_path_prefix=/path/to/output/prefix.tfrecord
CSVs with bounding box annotations and image metadata (including the image URLs)
can be downloaded from the Open Images GitHub repository:
https://github.com/openimages/dataset
This script will include every image found in the input_images_directory in the
output TFRecord, even if the image has no corresponding bounding box annotations
in the input_annotations_csv.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

import contextlib2
import pandas as pd
import tensorflow as tf

from object_detection.dataset_tools import oid_tfrecord_creation
from object_detection.utils import label_map_util

In [2]:
# all_annotations = pd.read_csv('/root/oid_per300_ckpt476120_eval_metrics/metrics.csv', header=None)

In [3]:
all_annotations = pd.read_csv('/root/oid_v4_label_map_validation_eval_metrics_modelv4/metrics.csv', names=['class', 'AP@0.5'])

In [4]:
# df=all_annotations
# df[df['class'].isin(list(ImageID_per300_set))]

In [5]:
df=all_annotations
df[df['class'].str.contains('Parking meter', regex=False)]

,class,AP@0.5
239,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/P...,NaN


In [6]:
df.iloc[224]['class']

'OpenImagesV2_PerformanceByCategory/AP@0.5IOU/Panda'

In [7]:
df=all_annotations
df[df['class'].str.contains('Plastic bag', regex=False)]

,class,AP@0.5
217,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/P...,NaN


In [8]:
df.iloc[202]['class']

'OpenImagesV2_PerformanceByCategory/AP@0.5IOU/Swim cap'

In [9]:
df=all_annotations
df[df['class'].str.contains('Coconut', regex=False)]

,class,AP@0.5
218,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/C...,NaN


In [10]:
df[df['class'].str.contains('Coconut', regex=False)].iloc[0]['AP@0.5']

nan

In [11]:
filtered_df = df[df['AP@0.5'].notnull()]['AP@0.5']



In [12]:
filtered_df.mean()

0.43269935664028225

In [13]:
df[df['AP@0.5'].isnull()]

,class,AP@0.5
1,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/Horn,NaN
3,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/Oboe,NaN
5,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/R...,NaN
10,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/F...,NaN
15,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/G...,NaN
17,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/C...,NaN
18,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/A...,NaN
24,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/W...,NaN
25,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/C...,NaN
26,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/H...,NaN


In [14]:
print(len(df[df['AP@0.5'].isnull()]))

345


In [23]:
print("afd{},dfdf{}".format(11,33))

afd11,dfdf33


In [24]:
import json
import sys


In [34]:
metric_dir = '/root/oid_bbox_trainable_label_map_validation_eval_metrics_faster_rcnn_inception_resnet_v2_atrous_oid'
# metric_output = sys.argv[2]


df = pd.read_csv(os.path.join(metric_dir ,'metrics.csv'), names=['class', 'AP@0.5'])

filtered_df = df[df['AP@0.5'].notnull()]['AP@0.5']
mAP_dict = {}
mAP_dict['mAP'] = filtered_df.mean()
# mAP_dict['Coconut'] = df[df['class'].str.contains('Coconut', regex=False)].iloc[0]['AP@0.5']
# mAP_dict['Parking meter'] = df[df['class'].str.contains('Parking', regex=False)].iloc[0]['AP@0.5']
mAP_dict['metric_path'] = os.path.join(metric_dir ,'metrics.csv')

##display
print("Eval for {}".format(mAP_dict['metric_path']))
print("mAP@0.5: {}".format(mAP_dict['mAP']))
# print("Coconut AP@0.5: {}".format(mAP_dict['Coconut']))
# print("Parking meter AP@0.5: {}".format(mAP_dict['Parking meter']))
print("Saving to: {}".format(os.path.join(metric_dir ,'metrics.json')))


Eval for /root/oid_bbox_trainable_label_map_validation_eval_metrics_faster_rcnn_inception_resnet_v2_atrous_oid/metrics.csv
mAP@0.5: 0.00105359674273
Saving to: /root/oid_bbox_trainable_label_map_validation_eval_metrics_faster_rcnn_inception_resnet_v2_atrous_oid/metrics.json


In [31]:
len(df[df['class'].str.contains('Coconut', regex=False)])

0

In [32]:
query_df = df[df['class'].str.contains('Coconut', regex=False)]
if len(query_df) > 0:
    mAP_dict['Coconut'] = query_df.iloc[0]['AP@0.5']
else:
    mAP_dict['Coconut'] = 'NOT EXIST'
query_df = df[df['class'].str.contains('Parking meter', regex=False)]
if len(query_df) > 0:
    mAP_dict['Parking meter'] = query_df.iloc[0]['AP@0.5']
else:
    mAP_dict['Parking meter'] = 'NOT EXIST'

In [33]:
mAP_dict

{'Coconut': 'NOT EXIST',
 'Parking meter': 'NOT EXIST',
 'mAP': 0.0010535967427278612,
 'metric_path': '/root/oid_bbox_trainable_label_map_validation_eval_metrics_faster_rcnn_inception_resnet_v2_atrous_oid/metrics.csv'}

In [ ]:
# compare models metric
oid_bbox_trainable_label_map_add2_validation_eval_metrics_train_out_547_8GPU8batch_frozen_575184


In [38]:
metric_dir = '/root/oid_bbox_trainable_label_map_add2_validation_eval_metrics_train_out_547_8GPU8batch_frozen_575184'
# metric_output = sys.argv[2]
df1 = pd.read_csv(os.path.join(metric_dir ,'metrics.csv'), names=['class', 'AP@0.5'])


In [40]:
metric_dir = '/root/oid_bbox_trainable_label_map_add2_validation_eval_metrics_train_out_547_7GPU7batch_frozen_476120_v2model_per300'
# metric_output = sys.argv[2]
df2 = pd.read_csv(os.path.join(metric_dir ,'metrics.csv'), names=['class', 'AP@0.5'])



In [41]:
df2

,class,AP@0.5
0,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/R...,0.395833
1,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/Horn,0.555871
2,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/Mug,0.666667
3,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/C...,0.791667
4,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/R...,0.038462
5,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/O...,0.455128
6,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/U...,0.611111
7,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/Hair,0.367405
8,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/S...,1.000000
9,OpenImagesV2_PerformanceByCategory/AP@0.5IOU/B...,0.327280
